## 1. 라이브러리 설치

In [ ]:

# 필요한 라이브러리 설치

# 기존 PaddlePaddle 완전 제거
%pip uninstall -y paddlepaddle paddlepaddle-gpu paddleocr

# ✅ PaddlePaddle 3.2.0 (CUDA 12.6+ 호환) + PaddleOCR 3.3.1
# 중요: PaddlePaddle 2.6.2는 PaddleOCR 3.x와 API 비호환!
# → PaddlePaddle 3.2.0으로 업그레이드 필수

# PaddlePaddle GPU 3.2.0 설치 (CUDA 12.6+ 호환)
%pip install paddlepaddle-gpu==3.2.0 -i https://www.paddlepaddle.org.cn/packages/stable/cu126/ -q

# 호환성 패키지 설치 (setuptools 오류 방지)
%pip install --upgrade setuptools backports.tarfile -q

# PaddleOCR 및 기타 라이브러리 설치
%pip install --upgrade paddleocr timm albumentations -q





In [ ]:
import os
import time
import random
import warnings
warnings.filterwarnings('ignore')

import timm
import torch
from paddleocr import PaddleOCR  # 🔥 PaddleOCR (빠른 한국어 OCR)
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import time  # 학습 시간 측정용
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW  # Adam → AdamW (Weight Decay)
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from scipy import stats  # K-Fold 앙상블용



## 2. 시드 고정

In [ ]:
# 시드 고정
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


## 3. Hyperparameters (EDA 결과 반영)

In [ ]:
# ========== 하이퍼파라미터 (Best Practice! 🔥) ========== #
CFG = {
    # 데이터
    'IMG_SIZE': 224,  # Swin Transformer 기본값
    'BATCH_SIZE': 16,  # Swin은 EfficientNet보다 가벼움
    'NUM_CLASSES': 17,
    
    # 학습
    'EPOCHS': 20,  # Fold당 20 Epoch
    'LR': 1e-4,  # Swin Transformer Fine-tuning LR
    'WEIGHT_DECAY': 0.01,  # 강화
    
    # 모델
    'MODEL_NAME': 'swin_base_patch4_window7_224',  # 🔥 Swin Transformer!
    'DROPOUT': 0.5,  # 강화 (Overfitting 해결)
    'LABEL_SMOOTHING': 0.1,  # 추가 (Regularization)
    
    # 🔥 OCR 설정
    'USE_OCR': True,
    'TEXT_EMB_DIM': 128,  # 텍스트 임베딩 차원
    'OCR_HEIGHT_RATIO': 0.25,  # 상단 25% (제목 영역)
    'OCR_MAX_LENGTH': 100,  # OCR 텍스트 최대 길이
    
    # K-Fold (중요!)
    'N_FOLDS': 5,  # 🔥 5-Fold Cross Validation!
    
    # 경로
    'BASE_DIR': '/home/realtheai/cv_competetion',
    'TRAIN_CSV': '/home/realtheai/cv_competetion/data/train.csv',
    'SAMPLE_SUBMISSION_CSV': '/home/realtheai/cv_competetion/data/sample_submission.csv',
    'TRAIN_IMG_DIR': '/home/realtheai/cv_competetion/data/train',
    'TEST_IMG_DIR': '/home/realtheai/cv_competetion/data/test',
    
    # 디바이스
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu',
}

print("\n" + "="*60)
print("📊 Configuration (08_Best_Practice)")
print("="*60)
print(f"모델: {CFG['MODEL_NAME']}")
print(f"IMG_SIZE: {CFG['IMG_SIZE']}")
print(f"BATCH_SIZE: {CFG['BATCH_SIZE']}")
print(f"EPOCHS: {CFG['EPOCHS']} (Fold당)")
print(f"K-Folds: {CFG['N_FOLDS']}")
print(f"Learning Rate: {CFG['LR']}")
print(f"Weight Decay: {CFG['WEIGHT_DECAY']}")
print(f"Dropout: {CFG['DROPOUT']}")
print(f"Label Smoothing: {CFG['LABEL_SMOOTHING']}")
print(f"Device: {CFG['DEVICE']}")
print("="*60)



## 4 OCR 초기화 및 텍스트 처리 함수

In [ ]:
# 🔥 PaddleOCR 초기화 (한국어 + GPU 자동 감지)
print("\n📖 PaddleOCR 초기화 중...")
print("   (처음 실행시 모델 다운로드 1~2분 소요)")

# PaddlePaddle GPU 사용 가능 여부 확인
import paddle
print(f"\n🔍 PaddlePaddle 버전: {paddle.__version__}")
print(f"   GPU 컴파일 여부: {paddle.device.is_compiled_with_cuda()}")
if paddle.device.is_compiled_with_cuda():
    print(f"   사용 가능한 GPU 수: {paddle.device.cuda.device_count()}")

# 로그 최소화
import logging
logging.getLogger('ppocr').setLevel(logging.ERROR)

# PaddleOCR 초기화 (한국어 특화)
# ✅ PaddleOCR 3.0: use_gpu 파라미터 제거됨, GPU 자동 감지!
ocr = PaddleOCR(
    use_angle_cls=True, 
    lang='korean'
    # use_gpu 파라미터 없음 → PaddlePaddle이 자동으로 GPU 감지!
)



def extract_text_from_title(image_path, ocr, height_ratio=0.25):

    try:
        # 이미지 로드
        img = Image.open(image_path).convert('RGB')
        img_array = np.array(img)
        
        # 상단 영역만 crop (제목 영역)
        height = img_array.shape[0]
        title_region = img_array[:int(height * height_ratio), :]
        
        # PaddleOCR 수행
        results = ocr.ocr(title_region, cls=True)
        
        # 텍스트만 추출하여 합치기
        if results and results[0]:
            texts = [line[1][0] for line in results[0]]
            combined_text = ' '.join(texts)
            return combined_text
        
        return ""
    
    except Exception as e:
        # OCR 실패시 빈 문자열 반환
        return ""


def text_to_simple_vector(text, max_length=100):

    vector = np.zeros(max_length, dtype=np.float32)
    for i, char in enumerate(text[:max_length]):
        # 문자 코드를 0~1로 정규화
        vector[i] = ord(char) / 1000.0
    return vector




## 5. Dataset 클래스

In [ ]:
class SwinOCRDataset(Dataset):

    def __init__(self, df, img_dir, ocr, transform=None, is_test=False):
        self.df = df
        self.img_dir = img_dir
        self.ocr = ocr
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['ID']
        img_path = os.path.join(self.img_dir, img_name)
        
        # 1. 이미지 로드
        img = np.array(Image.open(img_path).convert('RGB'))
        
        # 2. PaddleOCR로 텍스트 추출
        text = extract_text_from_title(img_path, self.ocr, CFG['OCR_HEIGHT_RATIO'])
        ocr_vector = text_to_simple_vector(text, CFG['OCR_MAX_LENGTH'])
        ocr_tensor = torch.FloatTensor(ocr_vector)
        
        # 3. 이미지 augmentation
        if self.transform:
            img = self.transform(image=img)['image']
        
        if self.is_test:
            return img, ocr_tensor
        
        target = self.df.iloc[idx]['target']
        return img, ocr_tensor, target



## 6. Augmentation 정의 (EDA 결과 반영)

In [ ]:
# 05_EDA 기반 Strong Augmentation
trn_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    
    # 🔥 05_EDA 핵심 발견: Test는 더 흐릿함! → Blur 강화
    A.GaussianBlur(blur_limit=(3, 9), p=0.7),  # 강하게!
    A.MotionBlur(blur_limit=7, p=0.4),
    
    # 품질 저하 시뮬레이션 (Test 데이터 특징)
    A.ImageCompression(quality_lower=60, p=0.4),
    A.Downscale(scale_min=0.7, scale_limit=0.95, p=0.3),
    A.GaussNoise(var_limit=(10, 50), p=0.4),
    
    # 기본 Augmentation
    A.Rotate(limit=3, p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=3, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.2, p=0.5),
    
    # 정규화
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

tst_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])


## 7. 모델 정의

In [ ]:
# 🔥 Swin Transformer + PaddleOCR Hybrid 모델
class SwinOCRClassifier(nn.Module):
    """
    Swin Transformer + PaddleOCR Hybrid Model
    
    구조:
    [OCR Vector] → [Linear] → [ReLU] → [128-dim]
                                                    → [Concat] → [FC] → [17 classes]
    [Image]      → [Swin Transformer] → [1024-dim]
    """
    def __init__(
        self, 
        model_name='swin_base_patch4_window7_224',
        num_classes=17,
        ocr_dim=100,
        dropout=0.5
    ):
        super().__init__()
        
        # 1. Swin Transformer (Backbone)
        self.backbone = timm.create_model(model_name, pretrained=True)
        
        # Swin의 출력 차원 확인 및 head 제거
        if hasattr(self.backbone, 'head'):
            in_features = self.backbone.head.in_features
            self.backbone.head = nn.Identity()
        elif hasattr(self.backbone, 'fc'):
            in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
        else:
            in_features = 1024  # Swin-Base 기본값
        
        # 2. OCR Feature Encoder
        self.ocr_encoder = nn.Sequential(
            nn.Linear(ocr_dim, 256),
            nn.ReLU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        
        # 3. Feature Fusion + Classifier
        fusion_dim = in_features + 128
        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, img, ocr_vec):
        """
        Args:
            img: [B, 3, 224, 224]
            ocr_vec: [B, 100]
        
        Returns:
            logits: [B, 17]
        """
        # 1. Image features (Swin Transformer)
        img_feat = self.backbone(img)
        
        # 2. OCR features
        ocr_feat = self.ocr_encoder(ocr_vec)
        
        # 3. Feature Fusion (Concatenation)
        fused_feat = torch.cat([img_feat, ocr_feat], dim=1)
        
        # 4. Classification
        output = self.classifier(fused_feat)
        return output



## 8. 데이터 로드

In [ ]:
# 데이터 로드
train_df = pd.read_csv(CFG['TRAIN_CSV'])
test_df = pd.read_csv(CFG['SAMPLE_SUBMISSION_CSV'])
meta_df = pd.read_csv(f"{CFG['BASE_DIR']}/data/meta.csv")

print(f"✅ 데이터 로드 완료")
print(f"   - Train: {len(train_df)}개")
print(f"   - Test: {len(test_df)}개")
print(f"   - 클래스: {CFG['NUM_CLASSES']}개")


## 9. 학습 함수 정의

In [ ]:
def train_epoch(model, loader, optimizer, criterion, scheduler, device):
    """1 Epoch 학습"""
    model.train()
    total_loss = 0
    
    pbar = tqdm(loader, desc="Training")
    for imgs, ocr_vecs, targets in pbar:
        imgs = imgs.to(device)
        ocr_vecs = ocr_vecs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs, ocr_vecs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / len(loader)


def validate(model, loader, criterion, device):
    """Validation"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for imgs, ocr_vecs, targets in tqdm(loader, desc="Validation"):
            imgs = imgs.to(device)
            ocr_vecs = ocr_vecs.to(device)
            targets = targets.to(device)
            
            outputs = model(imgs, ocr_vecs)
            loss = criterion(outputs, targets)
            
            preds = outputs.argmax(dim=1)
            
            total_loss += loss.item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    return avg_loss, accuracy, f1, all_preds, all_labels

print("✅ 학습 함수 정의 완료")


In [ ]:
# 🔥 Swin Transformer + PaddleOCR Hybrid 모델
class SwinOCRClassifier(nn.Module):
    """
    Swin Transformer + PaddleOCR Hybrid Model
    
    구조:
    [OCR Vector] → [Linear] → [ReLU] → [128-dim]
                                                    → [Concat] → [FC] → [17 classes]
    [Image]      → [Swin Transformer] → [1024-dim]
    """
    def __init__(
        self, 
        model_name='swin_base_patch4_window7_224',
        num_classes=17,
        ocr_dim=100,
        dropout=0.5
    ):
        super().__init__()
        
        # 1. Swin Transformer (Backbone) - forward_features 사용
        self.backbone = timm.create_model(model_name, pretrained=True, num_classes=0)
        
        # 🔥 num_classes=0 으로 설정하면 head 없이 features만 반환
        in_features = self.backbone.num_features
        
        print(f"✅ Swin Transformer Feature 차원: {in_features}")
        
        # 2. OCR Feature Encoder
        self.ocr_encoder = nn.Sequential(
            nn.Linear(ocr_dim, 256),
            nn.ReLU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        
        # 3. Feature Fusion + Classifier
        fusion_dim = in_features + 128
        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )
        
        print(f"✅ Classifier Input 차원: {fusion_dim} (Image {in_features} + OCR 128)")
        
        self._first_forward = True  # 디버깅용
    
    def forward(self, img, ocr_vec):
        """
        Args:
            img: [B, 3, 224, 224]
            ocr_vec: [B, 100]
        
        Returns:
            logits: [B, 17]
        """
        # 1. Image features (Swin Transformer)
        # num_classes=0 설정으로 자동으로 [B, num_features] 반환
        img_feat = self.backbone(img)
        
        # 🔥 디버깅: 첫 forward에서 shape 확인
        if self._first_forward:
            print(f"🔍 Debug - img_feat shape: {img_feat.shape}")
            self._first_forward = False
        
        # 2. OCR features
        ocr_feat = self.ocr_encoder(ocr_vec)
        
        # 3. Feature Fusion (Concatenation)
        fused_feat = torch.cat([img_feat, ocr_feat], dim=1)
        
        # 4. Classification
        output = self.classifier(fused_feat)
        return output



## 10. K-Fold Cross Validation

In [ ]:
# K-Fold Split
skf = StratifiedKFold(n_splits=CFG['N_FOLDS'], shuffle=True, random_state=SEED)

# Fold별 결과 저장
fold_results = []
oof_preds = np.zeros(len(train_df))
test_preds = np.zeros((len(test_df), CFG['N_FOLDS']))

print("\n" + "="*80)
print("🚀 5-Fold Cross Validation 시작!")
print("="*80)
print(f"총 예상 시간: 약 25시간 (Fold당 5시간)\n")

# K-Fold 학습
for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    print(f"\n{'='*80}")
    print(f"📁 Fold {fold + 1}/{CFG['N_FOLDS']}")
    print(f"{'='*80}")
    
    # Train/Val Split
    trn_df = train_df.iloc[train_idx].reset_index(drop=True)
    val_df = train_df.iloc[val_idx].reset_index(drop=True)
    
    print(f"   Train: {len(trn_df)}개 | Val: {len(val_df)}개")
    
    # Dataset
    trn_dataset = SwinOCRDataset(trn_df, CFG['TRAIN_IMG_DIR'], ocr, trn_transform)
    val_dataset = SwinOCRDataset(val_df, CFG['TRAIN_IMG_DIR'], ocr, tst_transform)
    
    # DataLoader
    trn_loader = DataLoader(
        trn_dataset,
        batch_size=CFG['BATCH_SIZE'],
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG['BATCH_SIZE'],
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    # Model
    model = SwinOCRClassifier(
        CFG['MODEL_NAME'],
        CFG['NUM_CLASSES'],
        CFG['OCR_MAX_LENGTH'],
        CFG['DROPOUT']
    ).to(CFG['DEVICE'])
    
    # Loss (Label Smoothing)
    criterion = nn.CrossEntropyLoss(label_smoothing=CFG['LABEL_SMOOTHING'])
    
    # Optimizer & Scheduler
    optimizer = AdamW(
        model.parameters(),
        lr=CFG['LR'],
        weight_decay=CFG['WEIGHT_DECAY']
    )
    
    total_steps = len(trn_loader) * CFG['EPOCHS']
    scheduler = CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=1e-6)
    
    # Training
    best_f1 = 0
    best_epoch = 0
    
    for epoch in range(1, CFG['EPOCHS'] + 1):
        start_time = time.time()
        
        # Train
        train_loss = train_epoch(model, trn_loader, optimizer, criterion, scheduler, CFG['DEVICE'])
        
        # Validate
        val_loss, val_acc, val_f1, val_preds_list, val_targets_list = validate(
            model, val_loader, criterion, CFG['DEVICE']
        )
        
        elapsed_time = time.time() - start_time
        
        # Best 모델 저장
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_epoch = epoch
            torch.save(model.state_dict(), f"{CFG['BASE_DIR']}/fold{fold+1}_best.pth")
            # OOF 예측 저장
            oof_preds[val_idx] = val_preds_list
            print(f"   ✅ Best 모델 저장! (F1: {best_f1:.4f})")
        
        print(f"\n[Epoch {epoch}/{CFG['EPOCHS']}] {elapsed_time:.1f}s")
        print(f"   Train Loss: {train_loss:.4f}")
        print(f"   Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")
        print(f"   Best F1: {best_f1:.4f} (Epoch {best_epoch})")
    
    # Fold 결과 저장
    fold_results.append({
        'fold': fold + 1,
        'best_f1': best_f1,
        'best_epoch': best_epoch
    })
    
    print(f"\n🎉 Fold {fold + 1} 완료!")
    print(f"   Best F1: {best_f1:.4f} (Epoch {best_epoch})")
    
    # Test 예측 (Best 모델)
    model.load_state_dict(torch.load(f"{CFG['BASE_DIR']}/fold{fold+1}_best.pth"))
    model.eval()
    
    test_dataset = SwinOCRDataset(test_df, CFG['TEST_IMG_DIR'], ocr, tst_transform, is_test=True)
    test_loader = DataLoader(
        test_dataset,
        batch_size=CFG['BATCH_SIZE'],
        shuffle=False,
        num_workers=4
    )
    
    fold_test_preds = []
    with torch.no_grad():
        for imgs, ocr_vecs in tqdm(test_loader, desc=f"Fold {fold+1} Test Inference"):
            imgs = imgs.to(CFG['DEVICE'])
            ocr_vecs = ocr_vecs.to(CFG['DEVICE'])
            outputs = model(imgs, ocr_vecs)
            preds = outputs.argmax(dim=1)
            fold_test_preds.extend(preds.cpu().numpy())
    
    test_preds[:, fold] = fold_test_preds
    
    print(f"   ✅ Fold {fold + 1} Test 예측 완료")

print("\n" + "="*80)
print("🎉 모든 Fold 학습 완료!")
print("="*80)


## K-fold 결과 요약

In [ ]:
# Fold별 결과 출력
print("\n📊 Fold별 결과:")
print("="*60)
for result in fold_results:
    print(f"Fold {result['fold']}: F1 = {result['best_f1']:.4f} (Epoch {result['best_epoch']})")

# 평균 F1
mean_f1 = np.mean([r['best_f1'] for r in fold_results])
std_f1 = np.std([r['best_f1'] for r in fold_results])

print("="*60)
print(f"평균 F1: {mean_f1:.4f} ± {std_f1:.4f}")
print("="*60)

# OOF (Out-of-Fold) 성능
oof_f1 = f1_score(train_df['target'], oof_preds, average='macro')
oof_acc = accuracy_score(train_df['target'], oof_preds)

print(f"\n🎯 OOF (Out-of-Fold) 성능:")
print(f"   Accuracy: {oof_acc:.4f}")
print(f"   F1 Score: {oof_f1:.4f}")
print(f"\n💡 OOF 성능이 실제 Leaderboard와 더 가깝습니다!")

# 5-Fold 앙상블 (다수결)
final_preds = stats.mode(test_preds, axis=1)[0].flatten().astype(int)

# 제출 파일 생성
submission = test_df.copy()
submission['target'] = final_preds
submission.to_csv(f"{CFG['BASE_DIR']}/submission.csv", index=False)



## 11. 학습 결과 시각화

In [ ]:
# K-Fold 결과 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Fold별 Best F1 Score 비교
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# (1) Fold별 Best F1 Score
fold_nums = [r['fold'] for r in fold_results]
fold_f1s = [r['best_f1'] for r in fold_results]
fold_epochs = [r['best_epoch'] for r in fold_results]

axes[0, 0].bar(fold_nums, fold_f1s, color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'])
axes[0, 0].axhline(y=np.mean(fold_f1s), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {np.mean(fold_f1s):.4f}')
axes[0, 0].set_title('Best F1 Score per Fold', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Fold')
axes[0, 0].set_ylabel('F1 Score')
axes[0, 0].set_ylim([min(fold_f1s) - 0.05, max(fold_f1s) + 0.05])
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3, axis='y')
for i, (f1, epoch) in enumerate(zip(fold_f1s, fold_epochs)):
    axes[0, 0].text(fold_nums[i], f1 + 0.01, f'{f1:.4f}\n(E{epoch})', 
                    ha='center', fontsize=10, fontweight='bold')

# (2) Fold별 Best Epoch
axes[0, 1].bar(fold_nums, fold_epochs, color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'])
axes[0, 1].axhline(y=np.mean(fold_epochs), color='red', linestyle='--', linewidth=2,
                   label=f'Mean: {np.mean(fold_epochs):.1f}')
axes[0, 1].set_title('Best Epoch per Fold', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Fold')
axes[0, 1].set_ylabel('Epoch')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3, axis='y')
for i, epoch in enumerate(fold_epochs):
    axes[0, 1].text(fold_nums[i], epoch + 0.3, f'{epoch}', 
                    ha='center', fontsize=11, fontweight='bold')

# (3) F1 Score 분포 (Box Plot)
axes[1, 0].boxplot([fold_f1s], labels=['All Folds'], widths=0.5)
axes[1, 0].scatter([1]*len(fold_f1s), fold_f1s, c=['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'], 
                   s=100, alpha=0.6, edgecolors='black', linewidth=1.5)
axes[1, 0].set_title('F1 Score Distribution', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('F1 Score')
axes[1, 0].grid(alpha=0.3, axis='y')
axes[1, 0].text(1.3, np.mean(fold_f1s), 
                f'Mean: {np.mean(fold_f1s):.4f}\nStd: {np.std(fold_f1s):.4f}\nOOF: {oof_f1:.4f}',
                fontsize=11, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# (4) 성능 요약 테이블
axes[1, 1].axis('off')
summary_data = [
    ['Metric', 'Value'],
    ['─' * 20, '─' * 20],
    ['Mean F1 (Folds)', f'{np.mean(fold_f1s):.4f}'],
    ['Std F1 (Folds)', f'{np.std(fold_f1s):.4f}'],
    ['Min F1', f'{min(fold_f1s):.4f}'],
    ['Max F1', f'{max(fold_f1s):.4f}'],
    ['─' * 20, '─' * 20],
    ['OOF F1 Score', f'{oof_f1:.4f}'],
    ['OOF Accuracy', f'{oof_acc:.4f}'],
    ['─' * 20, '─' * 20],
    ['Best Fold', f'Fold {fold_nums[np.argmax(fold_f1s)]}'],
    ['Worst Fold', f'Fold {fold_nums[np.argmin(fold_f1s)]}'],
    ['Mean Best Epoch', f'{np.mean(fold_epochs):.1f}'],
]

table = axes[1, 1].table(cellText=summary_data, cellLoc='left', loc='center',
                         colWidths=[0.6, 0.4])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2.5)

# 헤더 스타일
for i in range(2):
    table[(0, i)].set_facecolor('#3498db')
    table[(0, i)].set_text_props(weight='bold', color='white')

# OOF 행 강조
for i in range(2):
    table[(7, i)].set_facecolor('#2ecc71')
    table[(7, i)].set_text_props(weight='bold')
    table[(8, i)].set_facecolor('#2ecc71')
    table[(8, i)].set_text_props(weight='bold')

axes[1, 1].set_title('Performance Summary', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/kfold_results.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 K-Fold 결과 시각화 저장: {CFG['BASE_DIR']}/kfold_results.png")
print("\n📊 시각화 내용:")
print("   1. Fold별 Best F1 Score 비교")
print("   2. Fold별 Best Epoch")
print("   3. F1 Score 분포 (Box Plot)")
print("   4. 성능 요약 테이블 (OOF 포함)")

## 12. 틀린 이미지 분석 

In [ ]:
# 🔍 OOF (Out-of-Fold) 틀린 예측 분석
print("\n" + "="*80)
print("🔍 틀린 예측 분석 (OOF)")
print("="*80)

# OOF 예측과 실제 레이블 비교 (정수형으로 변환)
oof_preds_int = oof_preds.astype(int)
actual_labels = train_df['target'].values.astype(int)

wrong_mask = (oof_preds_int != actual_labels)
wrong_df = train_df[wrong_mask].copy()
wrong_df['predicted'] = oof_preds_int[wrong_mask]
wrong_df['actual'] = actual_labels[wrong_mask]

print(f"\n📊 전체 틀린 예측: {len(wrong_df)}개 / {len(train_df)}개 (정확도: {(1 - len(wrong_df)/len(train_df))*100:.2f}%)")

# 클래스별 오답 분석
print(f"\n📈 클래스별 오답 개수:")
wrong_by_class = wrong_df.groupby('actual').size().sort_values(ascending=False)
for class_id, count in wrong_by_class.items():
    class_id = int(class_id)
    class_name = meta_df[meta_df['target'] == class_id]['class_name'].values[0]
    total_in_class = (train_df['target'] == class_id).sum()
    error_rate = (count / total_in_class) * 100
    print(f"   Class {class_id:2d} ({class_name[:25]:25s}): {count:2d}/{total_in_class:3d} ({error_rate:5.1f}% 오답)")

# 가장 많이 혼동되는 클래스 쌍
print(f"\n🔥 가장 많이 혼동되는 클래스 쌍 (Top 10):")
confusion_pairs = wrong_df.groupby(['actual', 'predicted']).size().sort_values(ascending=False).head(10)
for idx, ((actual, pred), count) in enumerate(confusion_pairs.items(), 1):
    actual = int(actual)
    pred = int(pred)
    actual_name = meta_df[meta_df['target'] == actual]['class_name'].values[0]
    pred_name = meta_df[meta_df['target'] == pred]['class_name'].values[0]
    print(f"   {idx:2d}. Class {actual:2d} ({actual_name[:20]:20s}) → Class {pred:2d} ({pred_name[:20]:20s}): {count}개")

# 틀린 이미지 시각화 (최대 12개)
print(f"\n🖼️ 틀린 예측 샘플 시각화 (최대 12개)")
n_samples = min(12, len(wrong_df))
sample_indices = np.random.choice(len(wrong_df), n_samples, replace=False)
samples = wrong_df.iloc[sample_indices]

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for idx, (ax, (_, row)) in enumerate(zip(axes, samples.iterrows())):
    img_path = os.path.join(CFG['TRAIN_IMG_DIR'], row['ID'])
    img = Image.open(img_path).convert('RGB')
    
    actual = int(row['actual'])
    predicted = int(row['predicted'])
    actual_name = meta_df[meta_df['target'] == actual]['class_name'].values[0]
    pred_name = meta_df[meta_df['target'] == predicted]['class_name'].values[0]
    
    ax.imshow(img)
    ax.set_title(f"실제: {actual} ({actual_name[:15]})\n예측: {predicted} ({pred_name[:15]})",
                 fontsize=9, color='red', fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/wrong_predictions.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 틀린 예측 시각화 저장: {CFG['BASE_DIR']}/wrong_predictions.png")


## 13. Confusion Matrix

In [ ]:
# 🔍 OOF Confusion Matrix 시각화
print("\n" + "="*80)
print("📊 OOF Confusion Matrix")
print("="*80)

# OOF 예측으로 Confusion Matrix 생성
oof_preds_int = oof_preds.astype(int)
actual_labels = train_df['target'].values.astype(int)
cm = confusion_matrix(actual_labels, oof_preds_int)

# 시각화
fig, ax = plt.subplots(figsize=(16, 14))
im = ax.imshow(cm, cmap='Blues', interpolation='nearest')
ax.set_title('Confusion Matrix (OOF - 5-Fold)', fontsize=18, fontweight='bold', pad=20)

# 컬러바
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Count', fontsize=12)

# 축 레이블 (클래스 이름)
tick_labels = []
for i in range(CFG['NUM_CLASSES']):
    class_name = meta_df[meta_df['target']==i]['class_name'].values[0]
    # 클래스 이름 줄바꿈 처리 (긴 이름 대응)
    if len(class_name) > 15:
        class_name = class_name[:12] + '...'
    tick_labels.append(f"{i}\n{class_name}")

ax.set_xticks(range(CFG['NUM_CLASSES']))
ax.set_yticks(range(CFG['NUM_CLASSES']))
ax.set_xticklabels(tick_labels, rotation=45, ha='right', fontsize=9)
ax.set_yticklabels(tick_labels, fontsize=9)

ax.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax.set_ylabel('True Label', fontsize=14, fontweight='bold')

# 각 셀에 숫자 표시
threshold = cm.max() / 2
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        # 대각선(정답)은 녹색 텍스트, 오답은 기본 색상
        if i == j:
            color = 'darkgreen' if cm[i, j] > threshold else 'green'
            weight = 'bold'
        else:
            color = 'white' if cm[i, j] > threshold else 'black'
            weight = 'normal'
        
        # 0은 표시하지 않음 (깔끔하게)
        if cm[i, j] > 0:
            ax.text(j, i, str(cm[i, j]), 
                   ha='center', va='center',
                   color=color, fontsize=9, fontweight=weight)

plt.tight_layout()
plt.savefig(f"{CFG['BASE_DIR']}/confusion_matrix.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Confusion Matrix 저장: {CFG['BASE_DIR']}/confusion_matrix.png")

# 통계 정보
print(f"\n📊 Confusion Matrix 통계:")
print(f"   총 샘플: {cm.sum():.0f}개")
print(f"   정답 (대각선): {np.trace(cm):.0f}개")
print(f"   오답 (비대각선): {cm.sum() - np.trace(cm):.0f}개")
print(f"   정확도: {np.trace(cm) / cm.sum() * 100:.2f}%")

# 클래스별 정확도 (Recall)
print(f"\n📈 클래스별 Recall (정답률):")
for i in range(CFG['NUM_CLASSES']):
    class_name = meta_df[meta_df['target']==i]['class_name'].values[0]
    total = cm[i, :].sum()
    correct = cm[i, i]
    recall = (correct / total * 100) if total > 0 else 0
    print(f"   Class {i:2d} ({class_name[:25]:25s}): {correct:3.0f}/{total:3.0f} ({recall:5.1f}%)")

# 가장 많이 혼동되는 쌍 찾기
print(f"\n🔥 가장 많이 혼동된 클래스 쌍 (비대각선 Top 5):")
confusion_list = []
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        if i != j and cm[i, j] > 0:  # 대각선 제외
            confusion_list.append((i, j, cm[i, j]))

# 혼동 개수로 정렬
confusion_list.sort(key=lambda x: x[2], reverse=True)
for idx, (true_class, pred_class, count) in enumerate(confusion_list[:5], 1):
    true_name = meta_df[meta_df['target']==true_class]['class_name'].values[0]
    pred_name = meta_df[meta_df['target']==pred_class]['class_name'].values[0]
    print(f"   {idx}. Class {true_class:2d} ({true_name[:20]:20s}) → Class {pred_class:2d} ({pred_name[:20]:20s}): {count:.0f}개")

print("\n💡 Confusion Matrix 해석:")
print("   ✅ 대각선 (녹색): 정확하게 예측한 개수")
print("   ❌ 비대각선: 잘못 예측한 개수")